In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image


ModuleNotFoundError: No module named 'torch'

In [ ]:
# Constants
image_size = 64
batch_size = 32
# Do not change the code for load_images
# Function to load images (modified for PyTorch)
def load_images(directory, label, desired_shape=(3, 64, 64)):
    images = []
    labels = []
    transform = transforms.Compose([
        transforms.Resize((desired_shape[1], desired_shape[2])), # Assuming desired_shape is (channels, height, width)
        transforms.ToTensor()
    ])

    for filename in os.listdir(directory):
        if filename.lower().endswith(('.jpg', '.png')):
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path)
            img_tensor = transform(img)

            # Check if the shape of the image is what we desire
            if img_tensor.shape == desired_shape:
                images.append(img_tensor)
                labels.append(label)

    return torch.stack(images), torch.tensor(labels)

# Load training and testing data
train_cats, train_cats_labels = load_images('cat_dog_data/train/cat', 0)
train_dogs, train_dogs_labels = load_images('cat_dog_data/train/dog', 1)
test_cats, test_cats_labels = load_images('cat_dog_data/test/cat', 0)
test_dogs, test_dogs_labels = load_images('cat_dog_data/test/dog', 1)

# Combine cat and dog data
train_images = torch.cat([train_cats, train_dogs], dim=0)
train_labels = torch.cat([train_cats_labels, train_dogs_labels], dim=0)
test_images = torch.cat([test_cats, test_dogs], dim=0)
test_labels = torch.cat([test_cats_labels, test_dogs_labels], dim=0)

# Dataset and DataLoader
train_dataset = torch.utils.data.TensorDataset(train_images, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# CNN model
class SimpleCNN(nn.Module):

    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Instantiate some convolutional layers
        # write your code here.
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        # Instantiate the ReLU nonlinearity
        # write your code here.
        self.relu1 = nn.ReLU()
        # Instantiate a max pooling layer
        # write your code here.
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Instantiate a fully connected layer
        # write your code here.
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # Apply conv followed by relu, then in next line pool
        # write your code here.
        x = self.pool1(self.relu1(self.conv1(x)))
        
        # Apply conv followed by relu, then in next line pool
        # write your code here.
        x = self.pool2(self.relu2(self.conv2(x)))
        
        # Prepare the image for the fully connected layer
        # write your code here.
        x = x.view(-1, 32 * 16 * 16)
        
        # Apply the fully connected layer and return the sigmoid of the result
        # write your code here.
        x = nn.functional.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        
        return x 
    
model = SimpleCNN()

# Loss and optimizer
# write your code here.
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Using Adam optimizer with learning rate 0.001

model.train()
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{10}], Mini-batch [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0


# Training loop
for epoch in range(10):
    running_loss = 0.0
    for inputs, labels in train_loader:
        # write your code here.
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{10}], Mini-batch [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(test_images)
    predicted = (outputs.squeeze() > 0.5).float()
    accuracy = (predicted == test_labels).float().mean()
    print(f"Test accuracy: {accuracy.item()}")